In [6]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [7]:
#Loading the data
file_path = Path('./Output_Files/paces_bonks_2015.csv')
df_bonk = pd.read_csv(file_path)
df_bonk.head()

,Bib,Name,Age,M/F,City,State,Country,Pace (0-5K),Pace (5-10K),Pace (10-15K),Pace (15-20K),Pace (20-25K),Pace (25-30K),Pace (30-35K),Pace (35-40K),Overall Pace,Calculated Bonk
0,3,"Desisa, Lelisa",25,M,Ambo,NaN,ETH,2.94,3.0,3.05,3.11,3.13,3.18,3.20,2.93,4.93,0
1,4,"Tsegay, Yemane Adhane",30,M,Addis Ababa,NaN,ETH,2.94,3.0,3.05,3.10,3.13,3.17,3.20,2.94,4.97,0
2,11,"Kipyego, Bernard",28,M,Eldoret,NaN,KEN,2.94,3.0,3.06,3.09,3.13,3.18,3.21,3.15,5.00,0
3,10,"Korir, Wesley",32,M,Kitale,NaN,KEN,2.94,3.0,3.05,3.10,3.13,3.18,3.20,3.09,5.00,0
4,5,"Tola, Tadese",27,M,Addis Ababa,NaN,ETH,2.94,3.0,3.05,3.10,3.13,3.18,3.20,3.33,5.10,0


In [10]:
# Define the features set
X = df_bonk.copy()
X = df_bonk.drop(["Calculated Bonk","Pace (30-35K)","Pace (35-40K)"], axis=1)
X.head()

,Bib,Name,Age,M/F,City,State,Country,Pace (0-5K),Pace (5-10K),Pace (10-15K),Pace (15-20K),Pace (20-25K),Pace (25-30K),Overall Pace
0,3,"Desisa, Lelisa",25,M,Ambo,NaN,ETH,2.94,3.0,3.05,3.11,3.13,3.18,4.93
1,4,"Tsegay, Yemane Adhane",30,M,Addis Ababa,NaN,ETH,2.94,3.0,3.05,3.10,3.13,3.17,4.97
2,11,"Kipyego, Bernard",28,M,Eldoret,NaN,KEN,2.94,3.0,3.06,3.09,3.13,3.18,5.00
3,10,"Korir, Wesley",32,M,Kitale,NaN,KEN,2.94,3.0,3.05,3.10,3.13,3.18,5.00
4,5,"Tola, Tadese",27,M,Addis Ababa,NaN,ETH,2.94,3.0,3.05,3.10,3.13,3.18,5.10


In [11]:
# Define the target set
y = df_bonk["Calculated Bonk"].values

In [12]:
# Split into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 55)
Counter(y_train)

NameError: name 'Counter' is not defined

In [32]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [34]:
Counter(y_resampled)

Counter({0: 17247, 1: 17247})

In [44]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_resampled)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_resampled)
X_test_scaled = X_scaler.transform(X_test)

In [45]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [46]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_resampled)

In [47]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [48]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,5471,284
Actual 1,534,297


In [52]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,5471,284
Actual 1,534,297


NameError: name 'acc_score' is not defined

In [50]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.08586136, 0.02212528, 0.12876241, 0.13093767, 0.12618683,
       0.12635928, 0.2305554 , 0.14921175])

In [51]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.2305554015436891, 'Pace_25k'),
 (0.14921175097321657, 'Pace_20k_25k'),
 (0.13093767184487304, 'Pace_10k'),
 (0.12876240986851256, 'Pace_5k'),
 (0.1263592837577774, 'Pace_20k'),
 (0.12618683297657313, 'Pace_15k'),
 (0.08586136487386542, 'Age'),
 (0.022125284161492734, 'Gender')]